In [1]:
import pandas as pd
import os
from collections import Counter, defaultdict
import json
import itertools
import numpy as np
# load config
with open('config.json', 'r') as f:
    config = json.load(f)
cwd = os.getcwd()
os.chdir(config['REPODIR'])
import Utils as U
from Corpus import Corpus
os.chdir(cwd)

import pickle as pkl

In [38]:
suffix = "tiny"
# +
# with open('embedding_data_final.pkl', 'rb') as f:
#     embed = pkl.load(f)

# embed_df = pd.DataFrame(embed)
# # -


# embed_df = pd.DataFrame(embed)
data = U.load_file(f'data_vF_{suffix}.pkl', 'pkl', config['DATADIR'])
data_df = pd.DataFrame(data)

In [ ]:
# embed_df = embed_df.merge(data_df, how= 'left', left_on= 'passage_key', right_on = 'passage_key')
# embed_df = embed_df.dropna(subset=['author_id', 'sent_embedding'])

In [19]:
len(data_df) # Merge lost fidelity, nothing lost

6530

In [43]:

n_classes = data_df.author_id.nunique()

from sklearn.preprocessing import OneHotEncoder
# label_encoder=OneHotEncoder(sparse_output=False) Current version, have to do workaround 3 lines down for deprecated
label_encoder=OneHotEncoder()

y= label_encoder.fit_transform(data_df['Sub_C'])#.to_numpy(dtype='int32').reshape(1,-1)
y = y.toarray()
n_classes = data_df.author_id.nunique()

ValueError: Expected 2D array, got 1D array instead:
array=['Sonnets' 'Sonnets' 'Sonnets' ... nan nan nan].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [21]:
n_classes

6

In [22]:
data_df.groupby('author_id').agg({'passage_key':'count'})

,passage_key
author_id,
1082,4829
1775,210
3671,420
4224,240
4872,291
5722,540


In [23]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

# +
data_df = pd.DataFrame(data)
data_df.head()

data_df['joined_text'] = data_df['text'].apply(' '.join)
# -

text = data_df['joined_text'].to_numpy()
text_clean = [tokenizer.tokenize(t.lower()) for t in text]

In [24]:
data_df['clean_text'] = text_clean

X = data_df['joined_text']

In [25]:

test_size = 0.2
val_size = 0.2
random_state =699

X_train, X_test, y_train, y_test = U.train_test_split(X, y, test_size=test_size,
                                                        random_state=random_state,
                                                        stratify=y)

# Split train set into train and validation sets
X_train, X_val, y_train, y_val = U.train_test_split(X_train, y_train, test_size=val_size/(1-test_size),
                                                    random_state=random_state,
                                                    stratify=y_train)

In [26]:
X_val

833       ISABELLA. I would to heaven I had your poten...
2748        Hath every pelting river made so proud    ...
3737      Tra. A vengeance on your crafty withered hid...
3765    Ere twice the horses of the sunne shall bring ...
5877     "But you went to steal, you know," he said ge...
                              ...                        
2621      ARMADO. I will tell thee wonders.   JAQUENET...
3776      French E. Saue you good Madam     Hel. Madam...
2835      MRS. PAGE. If you go out in your own semblan...
2387    Be not disturb’d with my infirmity. If you be ...
4727    over the heads of residents in the constituenc...
Name: joined_text, Length: 1306, dtype: object

In [27]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import accuracy_score, f1_score

# Preprocess text data - remove stop_words
vectorizer = CountVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(X_train)
X_val = vectorizer.transform(X_val)
X_test = vectorizer.transform(X_test)

tfidf_vectorizer = TfidfVectorizer(
    max_df=0.95, min_df=2, stop_words="english"
)

In [28]:

# Train LDA model
lda = LatentDirichletAllocation(n_components=n_classes, random_state = random_state) 
                                #,max_iter=50, learning_method='online', random_state=42
lda.fit(X_train)

# Predict subject for documents in validation set
y_val_pred = lda.transform(X_val)

# Predict subject for documents in test set
y_test_pred = lda.transform(X_test)


In [29]:
y_val_pred = np.argmax(y_val_pred, 1)
y_test_pred = np.argmax(y_test_pred, 1)

In [35]:
data_df.Sub_C.unique()

array(['Sonnets', 'Adonis (Greek deity)', 'King of England', 'Greece',
       'Tragedies', 'Comedies', 'Triangles (Interpersonal relations)',
       'Princesses', 'Conflict of generations', 'Courtship',
       'Historical drama', 'Rome', 'Fathers and daughters', 'Trojan War',
       'Married women', 'Venice (Italy)', 'Kings and rulers', 'Egypt',
       'Athens (Greece)', 'Married people', 'Islands', nan, 'William',
       'Tragicomedy', 'British', 'Art', 'Social life and customs',
       'Netherlands', 'South African War', 'Juvenile fiction', 'Travel',
       '"Childrens stories"', 'Voyages and travels', 'Fiction',
       'German fiction', 'Ghost stories', 'Youth',
       'Truthfulness and falsehood', 'Christian life', 'Paris (France)',
       'Man-woman relationships', 'Jewelry theft', 'English', '1685-1688',
       'Treasure troves', 'Jacobite Rebellion', 'Middle Ages',
       'Intergenerational relations', '1914-1918', 'English drama',
       '20th century'], dtype=object)

In [30]:

print("F1 Val Score: " + str(f1_score(np.argmax(y_val, 1), y_val_pred, average ='weighted')))
print("F1 Test Score: " + str(f1_score(np.argmax(y_test,1), y_test_pred, average ='weighted')))

F1 Val Score: 0.20910775633117384
F1 Test Score: 0.20191374550837543


In [16]:
y_train = lda.transform(X_train)

In [17]:
y_index_train = np.apply_along_axis(np.argmax, 1, y_train)
y_index_val = np.apply_along_axis(np.argmax, 1, y_val)

y_lda_topics_train = [subjects[idx] for idx in y_index_train]
y_lda_topics_val = [subjects[idx] for idx in y_index_val]

In [18]:
y_lda_topics_train
y_lda_topics_val

['Short stories',
 'Juvenile fiction',
 'United States',
 'Short stories',
 'Poetry',
 'Short stories',
 'Adventure stories',
 'Juvenile fiction',
 'Poetry',
 'Juvenile fiction',
 'Juvenile fiction',
 'United States',
 'Short stories',
 'Short stories',
 'History',
 'Juvenile fiction',
 'Juvenile fiction',
 'Fiction',
 'Poetry',
 'United States',
 'Poetry',
 '19th century',
 'Short stories',
 'United States',
 'Poetry',
 'Juvenile fiction',
 'Adventure stories',
 'Fiction',
 'Short stories',
 'Juvenile fiction',
 'Fiction',
 'Short stories',
 'Short stories',
 'Juvenile fiction',
 'Short stories',
 'Poetry',
 'Juvenile fiction',
 '19th century',
 'Poetry',
 'Poetry',
 'Translations into English',
 'Fiction',
 'Juvenile fiction',
 'Short stories',
 '19th century',
 'History',
 '19th century',
 'Juvenile fiction',
 'Juvenile fiction',
 'United States',
 'Juvenile fiction',
 'Poetry',
 'Short stories',
 'Translations into English',
 'United States',
 'Poetry',
 'Short stories',
 'Translat

In [1]:
X_train.todense()

NameError: name 'X_train' is not defined

In [20]:


np.mean(train['topic'] == y_lda_topics_train)

0.05738029610312443

In [21]:
labels_val

(array([0, 1, 2, ..., 0, 0, 5]),
 Index(['Fiction', 'Juvenile fiction', 'Description and travel', 'Drama',
        'Short stories', 'Poetry', 'World War', 'Biography',
        'Detective and mystery stories', '19th century',
        'Translations into English', 'History', 'Juvenile literature',
        'Science fiction', 'English', 'Great Britain', 'Children', 'France',
        'Conduct of life', 'Adventure stories', 'American', 'United States',
        'English wit and humor', 'England', 'Social life and customs',
        'Love stories', 'Historical fiction', 'Young women',
        'Personal narratives'],
       dtype='object'))

In [77]:
pred_val

array([0, 0, 0, ..., 0, 0, 0])

In [22]:

# Train a classifier on the LDA topics
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(y_val, labels_val)

# Evaluate performance on the validation set
pred_val = clf.predict(y_val)
acc_val = accuracy_score(labels_val, pred_val)
print('Accuracy on validation set:', acc_val)

# Evaluate performance on the test set
pred_test = clf.predict(y_test)
acc_test = accuracy_score(labels_test, pred_test)
print('Accuracy on test set:', acc_test)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.